In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import time
import os, requests

In [11]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    
    service = Service(r"D:\GS_WebScarping\project_08_mag\chromedriver.exe")
    driver = Chrome(service=service, options=options)
    
    driver.get(path)

    
    # cookie
    try:
        cookie = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//button[@class = 'btn btn-accent-solid']")))
        if cookie:
            cookie.click()
        else:
            print("No cookies")
            
    except Exception as e:
        print("No cookies")
    
    # number of cars 
    try:
        num_cars = driver.find_element(By.CSS_SELECTOR, ".searchresults__header-count").text
        if num_cars:
            cars = int(num_cars)
            print(f"{cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"no cars found and error is {e}")


    
    results = []

    for i in range(1): # here we wil have 'cars'
        try:
            car_links = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class = 'searchresult searchresult--full pb-0']")))
            driver.execute_script("arguments[0].scrollIntoView();", car_links[i])
            
            time.sleep(2)
            car_links[i].click()
        
            details = {}
            
            # car name and location --> done
            try:
                car_det_main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/section[@class="mag__grey-bkg"]')))
        
                # car name and make --> done
                if car_det_main:
                    car_name = WebDriverWait(car_det_main, 10).until(EC.presence_of_element_located((By.XPATH, './/h1[@class="heading__vehicle"]'))).text.strip()
                    if car_name:
                        splited_data = car_name.split("\n")
                        details['Car_Make'] = splited_data[0]
                        details['Title'] = splited_data[1]
                    else:
                        details['Car_Make'] = "n/a"
                        details['Title'] = "n/a"
                else:
                    print("No name card")
                        
                        
                # location --> done
                auc_loc_card = WebDriverWait(car_det_main, 10).until(EC.presence_of_element_located((By.XPATH, './/span[@class="heading__vehicle"]'))).text.strip()
                if auc_loc_card:
                    details['Location'] = auc_loc_card
                else:
                    details['Location'] = "n/a"

            except Exception as e:
                print(f"No car name, make and auc loc")
                
                
            # imgs --> done
            # try:
            #     img_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//div[@class='lightcontainer light-buttons']")))
            #     if img_card:
            #         img_track_card = img_card.find_element(By.XPATH, ".//div[@class='slick-track']")
            #         if img_track_card:
            #             imgs = img_track_card.find_elements(By.XPATH, ".//div[@class='lightthumb slick-slide']/following::img")
            #             if imgs:
            #                 imgs_list = [img.get_attribute("src") for img in imgs ]
            #             imgs_str = ", ".join(imgs_list)
            #             details['Images'] = imgs_str
            #         else:
            #             details['Images'] = "na"
            # except Exception as e:
            #     print(f"No images found")  
                

            
            # basic details --> done  
            # try:
            #     details_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//div[@class = 'col-lg-6 pt-2 pb-4 pt-md-3 pb-md-5 py-lg-2 align-items-end']")))
            #     if details_card:
            #         details_row_card = details_card.find_element(By.XPATH, ".//div[@class = 'row data-row']")
            #         if details_row_card:
            #             detail_vals = details_row_card.find_elements(By.XPATH, ".//span[@class = 'data--end-vehicleGrade']")
            #             if detail_vals:
            #                 detail_vals_list = [vals.text.strip() for vals in detail_vals]

            #                 details["Reg"] = detail_vals_list[0]
            #                 details["VAT Status"] = detail_vals_list[1]
            #                 details["Model"] = detail_vals_list[2]
            #                 details["Fuel Type"] = detail_vals_list[3]
            #                 details["Transmission"] = detail_vals_list[4]
            #                 details["Mileage"] = detail_vals_list[5]
            #                 details["Grade"] = detail_vals_list[6]
            #                 details["Former_Owners"] = detail_vals_list[7]
            #                 details["Services"] = detail_vals_list[8]
            #                 details["V5"] = detail_vals_list[9]
            #                 details["First Reg'd"] = detail_vals_list[10]
            #                 details["MOT"] = detail_vals_list[11]
            #             else:
            #                 print("No data details found")
            #     else:
            #         print("no details tab found")
            # except Exception as e:
            #     print(f"no details tab ") 
                
            # date and time
            # try:
            #     date_card = driver.find_element(By.CSS_SELECTOR, ".data-details")
            #     if date_card:
            #         date_time = date_card.find_element(By.CSS_SELECTOR, ".heavy-title").text
            #         if date_time:
            #             details['Date&time'] = date_time
            #         else:
            #             details['Date&time'] = "n/a"
            #     else:
            #         print("No date time")
            # except Exception as e:
            #     print(f"No date time")
                    
                 
            
            # powered by --> remains
            try:    
                pow_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-lg-6"]'))) 
                if pow_card:
                    rows = pow_card.find_elements(By.XPATH, './/div[@class="data__cell col-md-12"]')
                    if rows:
                        for row in rows:
                            pow_lbl = row.find_element(By.TAG_NAME, 'strong').text.strip()
                            pow_val = row.find_element(By.XPATH, './/span[@class="data--end-vehicleGrade"]').text.strip()
                            if pow_lbl and pow_val:
                                details[pow_lbl] = pow_val
                            else:
                                print("No pow details") 
            except Exception as e:
                print("No pow details")              
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    # issue here
        
            # mileage hist and powered by
            # try:
            #     mil_hist_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-lg-6 motHistorySection"]')))
            #     mil_hdr = mil_hist_card.find_element(By.XPATH, './/div[@class="data-table--header"]')

            #     # Try to locate the "View all" link
            #     try:
            #         view_all_link = mil_hdr.find_element(By.LINK_TEXT, 'View all')
            #         driver.execute_script("arguments[0].scrollIntoView();", view_all_link)
            #         view_all_link.click()

            #         # Wait for the full MOT history popup to appear
            #         mil_full_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="jconfirm-content"]')))
            #         # find rows
            #         mill_rows = mil_full_card.find_elements(By.CSS_SELECTOR, ".row")
            #         for row in mill_rows:
            #             # Extract dates and values
            #             mil_dates = row.find_elements(By.XPATH, './/div[@class="col-6 text-right"]')
            #             mil_vals = row.find_elements(By.XPATH, './/div[@class="col-6 text-left"]')

            #             mil_dets_list = []
            #             if mil_dates and mil_vals:
            #                 for d, v in zip(mil_dates, mil_vals):
            #                     mil_details = {d.text.strip(): v.text.strip()}
            #                     mil_dets_list.append(mil_details)
            #                 details["Mot_Hist"] = mil_dets_list
            #             else:
            #                 details["Mot_Hist"] = "na"

            #     # Handle the case where "View all" is not present
            #     except Exception as e:
            #         print(f"No view all button present")
                    
                    
            #         mil_data_sec_card = mil_hist_card.find_element(By.XPATH, './/div[@class="col-lg-12"]')
            #         mil_row_data = mil_data_sec_card.find_element(By.XPATH, ".//div[@class='row data-row']")
            #         # driver.execute_script("arguments[0].scrollIntoView();", mil_row_data)
            #         # import time
            #         # time.sleep(2)
            #         if mil_row_data:
            #             # Extract data from the table
            #             mil_cols = mil_row_data.find_elements(By.XPATH, './/span[@class="my-md-0 flex-grow-1 flex-row d-flex data__info data--end-vehicleGrade"]/following::strong')
            #             mil_vals = mil_row_data.find_elements(By.XPATH, './/span[@class="data--end-vehicleGrade"]')
                        

            #             mil_dets_list = []
            #             for d, v in zip(mil_cols, mil_vals):
            #                 mil_details = {d.text.strip(): v.text.strip()}
            #                 mil_dets_list.append(mil_details)
            #             details['Mot_Hist'] = mil_dets_list
            #         else:
            #             print("No MOT history found")
            #             details['Mot_Hist'] = "na"

            # except Exception as e:
            #     print(f"No MOT history found and error is: {e}")
            
            

            
            
            # features
            # try:
            #     fet_main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-lg-6 pt-2 pb-4 pt-md-3 pb-md-5 py-lg-5"]')))
            #     driver.execute_script("arguments[0].scrollIntoView();", fet_main)
            #     # fet_data_card = fet_main.find_element(By.XPATH, '//div[@class="row data-row"]')
            #     if fet_main:
            #         # get data
            #         fet_data = fet_main.find_elements(By.XPATH,'.//div[@class="data__info"]')
            #         fet_data_list = [fet.text.strip() for fet in fet_data]
            #         details["Features"] = fet_data_list
            #     else:
            #         details["Features"] = "na"
            # except Exception as e:
            #     print(f"No features found")


            # reports
            # try:
            #     repo_main_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//section[@class='mt-3 mt-md-0']")))
                
            #     # Get the data row
            #     repo_data_row = repo_main_card.find_element(By.XPATH, ".//div[@class='row data-row']")
                
            #     if repo_data_row:
            #         # Get the card with links
            #         repo_links_card = repo_data_row.find_elements(By.XPATH, ".//div[@class='data__cell col-md-6']")
                    
            #         links_data = []  # List to hold dictionaries with text and links
                    
            #         for link_card in repo_links_card:
            #             # Find all links and corresponding text
            #             links = link_card.find_elements(By.XPATH, ".//a")
            #             texts = link_card.find_elements(By.XPATH, ".//span")
                        
            #             # Ensure both lists have the same length to avoid mismatches
            #             for i in range(min(len(links), len(texts))):
            #                 href = links[i].get_attribute("href")
            #                 text = texts[i].text.strip()
            #                 links_data.append({text: href})
                    
            #         # Append the links_data list to the details dictionary
            #         details['links'] = links_data

            # except Exception as e:
            #     print(f"No reports")

                            

            # remarks
            # try:
            #     # Wait for the main card to load
            #     rem_main_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//div[@class='col-lg-16 py-4 pt-md-5 py-lg-5']")))
                
            #     # Find the data table
            #     rem_data_table = rem_main_card.find_element(By.XPATH, ".//div[@class='data-table']")
                
            #     # Find the data row
            #     rem_data_row = rem_data_table.find_element(By.XPATH, ".//div[@class='row data-row']")
                
            #     # Check if data row exists and extract the text
            #     if rem_data_row:
            #         rem_data = rem_data_row.find_element(By.XPATH, ".//div[contains(@class, 'data_cell col-md-12')]")
            #         remarks_text = driver.execute_script("return arguments[0].textContent;", rem_data).strip()
            #         details['Remarks'] = remarks_text if remarks_text else "na"
            #     else:
            #         details['Remarks'] = "na"
            # except Exception as e:
            #     print(f"No remarks found and error is {e}")
            
            
            results.append(details)
            
            driver.back()
            driver.refresh()
            # go back 
            # try:
            #     go_bk = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//a[@class='main-header__back']")))
            #     if go_bk:
            #         driver.execute_script("arguments[0].scrollIntoView();", go_bk)
            #         import time
            #         time.sleep(2)
            #         go_bk.click()
            #     else:
            #         print("No go back button found")
            # except Exception as e:
            #     print("no back button")
            
            
            
            # driver.back()

        except Exception as e:
            print(f"No car link and error is {e}")
    # except Exception as e:
    #     print(f"No car links found and error is {e}")
            
            

                            
                            
  
    # results.append(details)
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("mag.csv", index=False)
    
    
    # time.sleep(2)
    
    driver.quit()

path = "https://www.mag.co.uk/SearchResults?saleId=595495&VehicleType=Car"
scarpe(path)

33 number of cars found
No pow details


## issues in the code
### 1) back page problem 
### some information is still remaining to be extracted
### Login info remains

In [12]:
df =pd.read_csv("mag.csv")
df

,Car_Make,Title,Location
0,Audi,A3 S Line Sp Edtn Tdi 138 - 1968cc,Rotherham
